In [12]:
import numpy as np
import pandas as pd
from plotly import express as px

In [ ]:
import os
os.getcwd()

In [ ]:
df = pd.read_csv('reg_data/train_data.csv', index_col=0, parse_dates=['date'])
df.reset_index(drop=True, inplace=True)

In [ ]:
df.dtypes

In [ ]:
df.date[34]

In [ ]:
# .sort_values(['tic','date'])[['tic','date',]
wat = df.groupby("tic", as_index=True)['close'].ewm(com=60, adjust=True).std()
wat = wat.droplevel(0)

In [ ]:
wat.index

In [ ]:
wat

In [ ]:
df["volatility"] = wat

In [ ]:
df

In [ ]:
px.line(df[df.tic == "BA"], x = 'date', y = "volatility", color = 'tic',  )

In [ ]:
px.line(df[df.tic == "BA"], x = 'date', y = "close", color = 'tic',  )

In [ ]:
delta = 0.9836065
delta/(1-delta)
I = 1000

w = [(1-delta) * (delta**i) * i for i in range(I)]

In [ ]:

data = {'r_t': np.random.randint(0, 100, I)} # Example returns between -0.5 and 0.5
df = pd.DataFrame(data)

df['gains'] = df['r_t'].cumsum()
df

In [ ]:
df['r_t_hat'] = df['r_t'].ewm(com=60, adjust=False).mean() 

In [ ]:
df['std_r_t'] = df['r_t'].ewm(com=60, adjust=True).std() 

In [ ]:
df['sig'] = np.sqrt( df['std_r_t'] * 261)

In [ ]:
df

In [ ]:
df["r_t"][:25]

In [ ]:
I = 1000

t = 23

def sig_sq(t, delta, df):
    r = df['r_t']
    rh = df['r_t_hat']
    sc = 261 # Scaling constant, assuming 261 trading days in a year
    return sc * sum([(1-delta) * (delta**i) * (r[t-1-i] - rh[t])**2 for i in range(t)])


df["volat"] = [ np.sqrt(sig_sq(t, delta, df)) for t in range(0, I) ]


In [ ]:
fig = px.line(df, y="volat")
fig#.add_trace(px.line(df, y="r_t").data[0])

In [ ]:
fig = px.scatter(df, y="sig", x = "volat")
fig

In [22]:
import pandas as pd
import numpy as np

class bla:

    def __init__(self):
        self.df = pd.read_csv("sample_df.csv", index_col='day')
        self.day = 345

    def calc_periods_returns(self, period, days):
        #data =  self.df[self.df.day == self.day] #, ['volatility', 'tic']]
        data = self.df.loc[self.day, ['date_idx', 'volatility', 'tic']].set_index('tic')

        data['ret'] = self.df.loc[self.day - days : self.day, ['ret','tic']].groupby('tic').sum()['ret']
        data['returns'] = data['ret'] / (data['volatility'] * np.sqrt(days))
        data.set_index('date_idx', inplace = True)
        self.df.loc[self.day, period] = data['returns'] 
        #print(self.df.loc[self.day])



    def doit(self, rng = [5]):
        periods = {'ret_1M': 21,
            'ret_2M': 42,
            'ret_3M': 63,
            'ret_1Y': 252} 
        for i in rng:
            self.day = i
            for p,d in periods.items():
                self.calc_periods_returns(p,d)    

In [4]:
import pandas as pd
import numpy as np

class bla:

    def __init__(self):
        self.df = pd.read_csv("sample_df.csv").set_index(['day','tic'], drop=False)

        self.result = []

        self.df.fillna({'volatility':1}, inplace=True)

        self.periods = ['ret_1Y', 'ret_3M', 'ret_2M', 'ret_1M'] #['ret_1M', 'ret_2M', 'ret_3M', 'ret_1Y']
        self.days = [252, 63, 42, 21]#  [21,42,63,252]
        self.time_scale = pd.DataFrame(np.sqrt([self.days]), columns = self.periods)

        self.df = self.df[['day', 'tic', 'open', 'high', 'low', 'close', 'volume', 'macd', 'rsi_30', 
                           'volatility', 'ret'] + self.periods ]

        #self.periods = pd.DataFrame([[21,42,63,252]], columns= ['ret_1M', 'ret_2M', 'ret_3M', 'ret_1Y'])
        self.df[self.periods] = 0.0

    def calc_periods_returns(self):

        s = np.array(self.days) 
        s = np.maximum(self.day - s, np.array([0]))

        # s = np.maximum(self.day - np.array(self.days),np.array([0]))

        #data = self.df.loc[(s, slice(None)), ['volatility', 'ret']].copy(deep=True)

        volatility = self.df.loc[(self.day, slice(None)), ['volatility']]
        volatility.columns = pd.RangeIndex(start=0, stop=1, step=1)
        norm = 1 / (volatility @ self.time_scale)
               
               
        addition = self.df.loc[self.day - 1,['ret','ret','ret','ret']] 
        addition.columns = self.periods
        
        
        
        tmp = {k:v for k,v in zip(s,self.periods)}
        lower = self.df.loc[(s, slice(None)),'ret']
        lower = lower.unstack(level=0, )
        lower.columns = lower.columns.map(tmp)

        addition -= lower

        addition.fillna(0.0, inplace= True)

        self.tracking += addition

        update = self.tracking * norm
        #update.index = update.index.set_levels([self.day -1],level=0)

        #self.df[self.periods].update(update)
        #self.df.loc[(self.day,slice(None)), periods] = self.tracking.loc[:,periods]  / (volatility * np.sqrt(days))
        #self.df.loc[self.day,self.periods] = update
        #self.result.append(update)
        return update

    def doit(self, rng = [5]):
        self.dim = self.df.tic.nunique()
        
        # self.tracking = {'ret_1M': np.array([0.0]*self.dim ),
        #         'ret_2M':  np.array([0.0]*self.dim ),
        #         'ret_3M':  np.array([0.0]*self.dim ),
        #         'ret_1Y':  np.array([0.0]*self.dim )} 
        
        self.tracking = self.df.loc[0,self.periods ]
        l = len(rng)

        for i in rng:
            print('\r line', i, " (", int(i/l)*100 , " %)" , end='')
            self.day = i
            self.data = self.df.loc[self.day, :]
            #for p,d in self.periods.items():
            upd = self.calc_periods_returns()
            self.data.loc[:,self.periods] = upd.loc[self.day]


In [5]:
pd.set_option('display.precision', 2)

In [6]:
B = bla()
B.df

day   tic      open      high       low     close   volume  \
day  tic                                                                 
0    CL=F     0  CL=F     43.72     46.74     41.05     46.34   184679   
     ES=F     0  ES=F    900.00    932.75    893.50    925.50  1095813   
     NG=F     0  NG=F      5.55      6.09      5.50      5.97    41110   
     YM=F     0  YM=F   8718.00   9019.00   8715.00   8958.00    88889   
     ZS=F     0  ZS=F    965.00    985.75    963.00    970.00    14287   
...         ...   ...       ...       ...       ...       ...      ...   
2890 NG=F  2890  NG=F      1.70      1.78      1.69      1.75   186911   
     YM=F  2890  YM=F  25479.00  25772.00  25301.00  25689.00   200320   
     ZS=F  2890  ZS=F    865.00    891.00    864.00    884.25    10148   
     ZT=F  2890  ZT=F    110.29    110.29    110.28    110.28   302096   
     ZW=F  2890  ZW=F    483.75    495.50    478.50    490.00     2721   

             macd  rsi_30  volatility        ret  ret_1Y  ret_3M  ret_2M  \
day  tic                                                                   
0    CL=F    0.00  100.00        1.00    3123.84     0.0     0.0     0.0   
     ES=F    0.00  100.00        1.00       0.00     0.0     0.0     0.0   
     NG=F    0.00  100.00        1.00       0.00     0.0     0.0     0.0   
     YM=F    0.00  100.00        1.00  891800.00     0.0     0.0     0.0   
     ZS=F    0.00  100.00        1.00       0.00     0.0     0.0     0.0   
...           ...     ...         ...        ...     ...     ...     ...   
2890 NG=F   -0.04   50.20        0.27       0.00     0.0     0.0     0.0   
     YM=F  123.66   52.10     2214.70       0.00     0.0     0.0     0.0   
     ZS=F    6.00   58.13       27.22       0.00     0.0     0.0     0.0   
     ZT=F    0.02   61.25        1.08       0.00     0.0     0.0     0.0   
     ZW=F   -8.41   44.13       27.10       0.00     0.0     0.0     0.0   

           ret_1M  
day  tic           
0    CL=F     0.0  
     ES=F     0.0  
     NG=F     0.0  
     YM=F     0.0  
     ZS=F     0.0  
...           ...  
2890 NG=F     0.0  
     YM=F     0.0  
     ZS=F     0.0  
     ZT=F     0.0  
     ZW=F     0.0  

[20237 rows x 15 columns]

In [7]:
tics = B.df.tic.unique()

wat = B.df.loc[(slice(1,3), tics[:3]),['day', 'tic', 'open', 'high', 'low', 'close', 'volume', 'macd', 'rsi_30', 'volatility']]

In [34]:
fp = "/home/michael/Dropbox/Studies/M.Sc./Year2_SemesterB/65026 - Robotics and Reinforcement Learning/Project/futures_tics.md"
wat.round(2).to_markdown(fp, index=False)

In [ ]:
t = [f"'{t}': ''" for t in tics]
print(",\n".join(t))

'CL=F': '',
'ES=F': '',
'NG=F': '',
'YM=F': '',
'ZS=F': '',
'ZT=F': '',
'ZW=F': ''


In [8]:
!pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
tics = \
{'CL=F': 'Crude Oil',  
'ES=F': 'E-Mini S&P 500',  
'NG=F': 'Natural Gas',  
'YM=F': 'Mini Dow Jones Indus.-$5',  
'ZS=F': 'Soybean Futures',  
'ZT=F': '2-Year T-Note Futures',  
'ZW=F': 'Chicago SRW Wheat Futures' }
tics

{'CL=F': 'Crude Oil',
 'ES=F': 'E-Mini S&P 500',
 'NG=F': 'Natural Gas',
 'YM=F': 'Mini Dow Jones Indus.-$5',
 'ZS=F': 'Soybean Futures',
 'ZT=F': '2-Year T-Note Futures',
 'ZW=F': 'Chicago SRW Wheat Futures'}

In [3]:
bab = [{"tic": k, "name": v} for k, v in tics.items()]

In [33]:
INDICATORS = {
    "macd": "Moving Average Convergence Divergence",
    "boll_ub": "Bollinger Upper Band",
    "boll_lb": "Bollinger Lower Band",
    "rsi_30": "Relative Strength Index",
    "cci_30": "Commodity Channel Index",
    "dx_30": "Directional Movement Index",
    "close_30_sma": "Simple Moving Average",
    "close_60_sma": "Simple Moving Average "
}

In [34]:
import io

str_buffer = io.StringIO()



In [2]:
import pandas as pd

pd.DataFrame(INDICATORS, index=[1]).T.reset_index().to_markdown(str_buffer, index=False )
csv_string = str_buffer.getvalue()

print(csv_string)

NameError: name 'INDICATORS' is not defined

| index        | 1                                     |
|:-------------|:--------------------------------------|
| macd         | Moving Average Convergence Divergence |
| boll_ub      | Bollinger Upper Band                  |
| boll_lb      | Bollinger Lower Band                  |
| rsi_30       | Relative Strength Index               |
| cci_30       | Commodity Channel Index               |
| dx_30        | Directional Movement Index            |
| close_30_sma | Simple Moving Average                 |
| close_60_sma | Simple Moving Average                 |

In [82]:
bm = pd.read_csv("futures_data/backtest_metrics.csv")
bm

,type,algo,Final value,Ann. return,Ann. std,Sharpe,Max drawdown
0,Actor-Critic,a2c,1.282642e+06,0.206604,0.174913,1.161589,-0.212082
1,Actor-Critic,ddpg,1.460895e+06,0.331078,0.278087,1.166875,-0.211325
2,Actor-Critic,ppo,1.445195e+06,0.320271,0.137569,2.089502,-0.094702
3,Actor-Critic,td3,1.436106e+06,0.314001,0.124490,2.256948,-0.067516
4,Actor-Critic,sac,1.376750e+06,0.272814,0.137217,1.827375,-0.091481
5,Baseline,mvo,1.232968e+06,0.168870,0.076533,2.077666,-0.051423
6,Baseline,dji,1.388401e+06,0.296989,0.141187,1.913219,-0.089308


In [83]:
px.scatter(bm, y='Final value', x='Sharpe', color='type',   size='Ann. return', # hover_name='tic',
           title='Backtest Metrics for RL algorithms', text='algo',
           labels={'Final value': 'Final value', 'Sharpe': 'Sharpe Ratio'}, #, 'total_trades': 'Total Trades'
           width=600, height=600, 
          ).show() # template='plotly_white'

In [84]:
bm['Max drawdown'] = (bm['Max drawdown'] + 1) **2

In [85]:
bm

,type,algo,Final value,Ann. return,Ann. std,Sharpe,Max drawdown
0,Actor-Critic,a2c,1.282642e+06,0.206604,0.174913,1.161589,0.620815
1,Actor-Critic,ddpg,1.460895e+06,0.331078,0.278087,1.166875,0.622008
2,Actor-Critic,ppo,1.445195e+06,0.320271,0.137569,2.089502,0.819564
3,Actor-Critic,td3,1.436106e+06,0.314001,0.124490,2.256948,0.869526
4,Actor-Critic,sac,1.376750e+06,0.272814,0.137217,1.827375,0.825407
5,Baseline,mvo,1.232968e+06,0.168870,0.076533,2.077666,0.899798
6,Baseline,dji,1.388401e+06,0.296989,0.141187,1.913219,0.829360


In [86]:
px.scatter(bm, y='Ann. std', x='Ann. return', color='type',   size='Max drawdown', # hover_name='tic',
           title='Backtest Metrics for RL algorithms<br><sup>size represents Max drawdown</sup>', text='algo', 
           #labels={'Final value': 'Final value', 'Sharpe': 'Sharpe Ratio'}, #, 'total_trades': 'Total Trades'
           width=600, height=600, 
          ).show() # template='plotly_white'

In [2]:
import stockstats as ss

In [15]:
from stockstats import wrap

In [16]:
ssdf = wrap(B.df)

In [17]:
ssdf

day   tic      open      high       low     close   volume  \
day  tic                                                                 
0    CL=F     0  CL=F     43.72     46.74     41.05     46.34   184679   
     ES=F     0  ES=F    900.00    932.75    893.50    925.50  1095813   
     NG=F     0  NG=F      5.55      6.09      5.50      5.97    41110   
     YM=F     0  YM=F   8718.00   9019.00   8715.00   8958.00    88889   
     ZS=F     0  ZS=F    965.00    985.75    963.00    970.00    14287   
...         ...   ...       ...       ...       ...       ...      ...   
2890 NG=F  2890  NG=F      1.70      1.78      1.69      1.75   186911   
     YM=F  2890  YM=F  25479.00  25772.00  25301.00  25689.00   200320   
     ZS=F  2890  ZS=F    865.00    891.00    864.00    884.25    10148   
     ZT=F  2890  ZT=F    110.29    110.29    110.28    110.28   302096   
     ZW=F  2890  ZW=F    483.75    495.50    478.50    490.00     2721   

             macd  rsi_30  volatility        ret  ret_1y  ret_3m  ret_2m  \
day  tic                                                                   
0    CL=F    0.00  100.00        1.00    3123.84     0.0     0.0     0.0   
     ES=F    0.00  100.00        1.00       0.00     0.0     0.0     0.0   
     NG=F    0.00  100.00        1.00       0.00     0.0     0.0     0.0   
     YM=F    0.00  100.00        1.00  891800.00     0.0     0.0     0.0   
     ZS=F    0.00  100.00        1.00       0.00     0.0     0.0     0.0   
...           ...     ...         ...        ...     ...     ...     ...   
2890 NG=F   -0.04   50.20        0.27       0.00     0.0     0.0     0.0   
     YM=F  123.66   52.10     2214.70       0.00     0.0     0.0     0.0   
     ZS=F    6.00   58.13       27.22       0.00     0.0     0.0     0.0   
     ZT=F    0.02   61.25        1.08       0.00     0.0     0.0     0.0   
     ZW=F   -8.41   44.13       27.10       0.00     0.0     0.0     0.0   

           ret_1m  
day  tic           
0    CL=F     0.0  
     ES=F     0.0  
     NG=F     0.0  
     YM=F     0.0  
     ZS=F     0.0  
...           ...  
2890 NG=F     0.0  
     YM=F     0.0  
     ZS=F     0.0  
     ZT=F     0.0  
     ZW=F     0.0  

[20237 rows x 15 columns]

In [28]:
B.df["volatility_sqrt"] = np.sqrt(B.df["volatility"])

In [29]:
px.line(B.df, x = 'day', y = 'volatility_sqrt', color = 'tic')

In [31]:
px.line(B.df, x = 'day', y = 'close', color = 'tic')

In [27]:
px.line(B.df, x = 'day', y = 'ret', color = 'tic')

In [ ]:
B.df.columns

In [ ]:
what = B.df.index.get_level_values('day').unique()[1:]
what

In [ ]:

B.doit(what)
#B.doit(list(range(67,100)))

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f bla.calc_periods_returns B.doit(what)

In [ ]:
B.df

In [ ]:
B.df.index

In [ ]:
lal = B.df.xs('ZT=F',level='tic')

In [ ]:
lal

In [ ]:
from plotly import express as px
px.line(lal, y = 'ret_1M')

In [ ]:
a =  list(range(30,10,-1))
a

In [ ]:
sm = 0

w = 6
for e in range(1,20): 
    s = max(0,e-w)

    sm -= a[s]
    sm += a[e-1]
    bad = sum(a[s+1:e])
    print(e, s, e, a[s+1:e], bad, sm)

In [ ]:
sm = 0

w = 6
for e in range(1,len(a)): 
    s = e-w 
    if (s >= 0):
        sm -= a[s]
    
    sm += a[e-1]
    bad = a[max(0,s+1):e]
    print(e, s, e, bad, sum(bad), sm)

In [ ]:

def step(a, e, w, sm):
    s = e-w 
    if (s >= 0):
        sm -= a[s]
    
    sm += a[e-1]
    bad = a[max(0,s+1):e]
    print(e, s, e, bad, sum(bad), sm)
    return sm


sm = 0

w = 6
for e in range(1,len(a)): 
    sm = step(a, e, w, sm)

In [ ]:
sum([24, 23, 22, 21, 20, 19]) 

# Analyzing results

In [ ]:
import pandas as pd
from plotly import express as px

def read_backtest(path, label ):
    res_df = pd.read_csv(path)
    long_df = res_df.melt(id_vars=['date'], value_vars=['a2c', 'ddpg', 'ppo', 'td3', 'sac', 'mvo', 'dji'],
                      var_name="algo", value_name="money")
    long_df["label"] = label
    return long_df

df = read_backtest("futures_data/results/20250704-2310/backtest.csv", "FuturesTradingEnv")
df

In [17]:
df2 = read_backtest("futures_data/results/20250704-2310/backtest.csv", "StockTradingEnv")


In [20]:
long_df = pd.concat([df,df2])
long_df

,date,algo,money,label
0,2020-07-01,a2c,1.000000e+06,FuturesTradingEnv
1,2020-07-02,a2c,9.999745e+05,FuturesTradingEnv
2,2020-07-06,a2c,9.999960e+05,FuturesTradingEnv
3,2020-07-07,a2c,9.999726e+05,FuturesTradingEnv
4,2020-07-08,a2c,9.999622e+05,FuturesTradingEnv
...,...,...,...,...
2347,2021-10-22,dji,1.386322e+06,StockTradingEnv
2348,2021-10-25,dji,1.388813e+06,StockTradingEnv
2349,2021-10-26,dji,1.389427e+06,StockTradingEnv
2350,2021-10-27,dji,1.379083e+06,StockTradingEnv


In [21]:
px.line(long_df, x = 'date', y = 'money', color = 'algo', facet_col='label')